In [1]:
import gzip
import json

In [2]:
# Opening dataset
corpus = list()
with gzip.open('ecommerce.json.gz') as fp:
    for line in fp:
        entry = line.decode('utf8')
        corpus.append(json.loads(entry))

In [3]:
from pprint import pprint
pprint(corpus[15])

{'_id': 8691927,
 'cat': ' Suplementos e Vitaminas',
 'descr': 'O Power Óleo de Cártamo da Power Foods, chega ao mercado com alta '
          'concentração em suas capsulas com 1000mg/caps, a absorção delas é '
          'super-rápidas com a tecnologia Softgel e o melhor ainda, contem '
          'vitamina E.O Óleo, possui ácido linoleico, mais conhecido como '
          'Omega 6, e também acido oleico conhecido como Omega 9. Esses são '
          'ácidos graxos essenciais, ou seja aqueles que o nosso corpo não '
          'produz e que devem ser adquiridos por meio da dieta alimentar, '
          'contribuindo assim no controle da produção do LDL (colesterol ruim) '
          'do organismo, tudo isso reunido em uma única capsulas irá lhe '
          'proporcionar a queima de gordura localizada.O óleo de Cártamo é '
          'derivado de uma planta asiática, parente do girassol. Suas sementes '
          'atuam no organismo destruindo a gordura localizada, principalmente '
          '

In [4]:
# Building a classifier for products and categories (considering only the first 10 thousand products)
dataset = list()
for entry in corpus[:10000]:
    if 'cat' in entry:
        dataset.append( (entry['name'], entry['cat'].lower().strip()) )

In [5]:
len(dataset)

9953

In [6]:
# How many distinct categories do we have and how many items per category?
from collections import Counter
counter = Counter([cat for prod, cat in dataset])
pprint(counter.most_common())

[('bebês', 1208),
 ('eletroportáteis', 1052),
 ('automotivo', 915),
 ('utilidades domésticas', 857),
 ('suplementos e vitaminas', 787),
 ('ar-condicionado e aquecedores', 754),
 ('informática', 706),
 ('cama, mesa e banho', 670),
 ('tv e home theater', 644),
 ('perfumaria', 532),
 ('beleza e saúde', 497),
 ('dvds e blu-ray', 433),
 ('relógios', 410),
 ('pet shop', 391),
 ('instrumentos musicais', 44),
 ('celulares e telefones', 18),
 ('eletrodomésticos', 16),
 ('áudio', 13),
 ('alimentos e bebidas', 2),
 ('livros', 2),
 ('brinquedos', 1),
 ('linha industrial', 1)]


# Building an SVM Classifier with Bag of Words

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

In [8]:
import nltk
stopwords = nltk.corpus.stopwords.words('portuguese')

In [9]:
# Building the SVM model with Pipeline
modelo = Pipeline([('vect', TfidfVectorizer()), ('clf', SVC(kernel = 'linear', probability = True))])

In [10]:
# Object for Normalizate of labels
encoder = LabelEncoder()

In [11]:
# Getting data and labels
data = [prod for prod, cat in dataset]
labels = [cat for prod, cat in dataset]
len(data)

9953

In [12]:
# Normalization of labels
target = encoder.fit_transform(labels)

In [13]:
# Items
encoder.classes_.item(1)

'ar-condicionado e aquecedores'

In [14]:
# Fit Model
modelo.fit(data, target)

Pipeline(steps=[('vect', TfidfVectorizer()),
                ('clf', SVC(kernel='linear', probability=True))])

In [15]:
# Predicting the category from the description
modelo.predict(["Refrigerador Brastemp com função frostfree"])

array([9], dtype=int64)

In [16]:
# Predicting the category from the description
print (encoder.classes_[9])

eletrodomésticos


In [17]:
# Probabilities of a product
probs = modelo.predict_proba(["Ventilador"])

In [18]:
#  Category probabilities for the Fan object
guess = [(class_, probs.item(n)) for n, class_ in enumerate(encoder.classes_)]
pprint(guess)

[('alimentos e bebidas', 1.490857218834176e-07),
 ('ar-condicionado e aquecedores', 1.696332007986466e-07),
 ('automotivo', 1.7197168037992986e-07),
 ('bebês', 3.431034662792874e-07),
 ('beleza e saúde', 1.7643548436633242e-07),
 ('brinquedos', 1.7510481133682117e-05),
 ('cama, mesa e banho', 1.7951112441902603e-07),
 ('celulares e telefones', 1.1905604906060375e-05),
 ('dvds e blu-ray', 2.2257574857902048e-07),
 ('eletrodomésticos', 7.946287445013626e-07),
 ('eletroportáteis', 0.999956726892783),
 ('informática', 2.2652959681312867e-07),
 ('instrumentos musicais', 7.482884027934183e-07),
 ('linha industrial', 5.654693879164747e-06),
 ('livros', 5.755712552858329e-07),
 ('perfumaria', 2.0092098850058296e-07),
 ('pet shop', 3.0423491213958033e-07),
 ('relógios', 1.3115958848755237e-06),
 ('suplementos e vitaminas', 4.6267559240100573e-07),
 ('tv e home theater', 2.706444814785229e-07),
 ('utilidades domésticas', 1.735511550912569e-07),
 ('áudio', 1.7213698578035609e-06)]


In [19]:
# Adjusted probability of categories for the Fan object
from operator import itemgetter
for cat, proba in sorted(guess, key = itemgetter(1), reverse = True):
    print ('{}: {:.4f}'.format(cat, proba))

eletroportáteis: 1.0000
brinquedos: 0.0000
celulares e telefones: 0.0000
linha industrial: 0.0000
áudio: 0.0000
relógios: 0.0000
eletrodomésticos: 0.0000
instrumentos musicais: 0.0000
livros: 0.0000
suplementos e vitaminas: 0.0000
bebês: 0.0000
pet shop: 0.0000
tv e home theater: 0.0000
informática: 0.0000
dvds e blu-ray: 0.0000
perfumaria: 0.0000
cama, mesa e banho: 0.0000
beleza e saúde: 0.0000
utilidades domésticas: 0.0000
automotivo: 0.0000
ar-condicionado e aquecedores: 0.0000
alimentos e bebidas: 0.0000
